# 1. RockAuto

In [12]:
import pandas as pd

input_ = pd.read_excel('./crawler.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import json

for i in tqdm(range(len(input_)), desc='Progress', ncols=77):
    dict_ = json.loads(input_.loc[i, 'Json_Specification'])
    for value in dict_.values():
        if len(list(value.values())) == 1:
            input_.loc[i, list(value.values())[0]] = True
        else:
            input_.loc[i, list(value.values())[0]] = list(value.values())[1]

input_.to_excel('./crawler.xlsx', index=False)

input_

Progress: 100%|█████████████████████████████| 39/39 [00:00<00:00, 528.40it/s]


,No,Group,Type,Type Code,Manufacturer,Part Number,Note 1,Json_Note 2,Choose,Json_Price,...,Travel Length (IN),Upper Mount Code,Upper Mount Type,Dust Shield Included,Body Diameter (IN),Body Length (IN),Finish,Rod Diameter (mm),Shock Absorber Bore Size (MM),Foam Cell Design
0,1,Steering,Steering Damper,7412,RANCHO,RS5401,RS5000,"{""0"": ""Category: Steering Damper""}",,"{""automatic"": ""Add to Cart to See Price""}",...,7.18,L1,"Loop Bushing Mount - 5/8"" Bushing ID X 1.5/16""...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Steering,Steering Damper,7412,RANCHO,RS5402,RS5000; Bracket/Hardware not Included,"{""0"": ""Category: Steering Damper""}",,"{""automatic"": ""Add to Cart to See Price""}",...,8.02,S4,"Stem Mount - 2.3/8"" Stem Length X 3/8""-16 Thre...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Steering,Steering Damper,7412,RANCHO,RS5403,RS5000; Single,"{""0"": ""Category: Steering Damper""}",,"{""automatic"": ""Add to Cart to See Price""}",...,9.03,S4,"Stem Mount - 2.3/8"" Stem Length X 3/8""-16 Thre...",Yes,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Steering,Steering Damper,7412,RANCHO,RS5404,RS5000; w/O.E. Steering Stabilizer; Bolts To O...,"{""0"": ""Category: Steering Damper""}",,"{""automatic"": ""Add to Cart to See Price""}",...,9.44,L1,"Loop Bushing Mount - 5/8"" Bushing ID X 1.5/16""...",Yes,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Steering,Steering Damper,7412,RANCHO,RS5405,RS5000; Dual Steering,"{""0"": ""Category: Steering Damper""}",,"{""automatic"": ""Add to Cart to See Price""}",...,8.92,S4,"Stem Mount - 2.3/8"" Stem Length X 3/8""-16 Thre...",Yes,NaN,NaN,NaN,NaN,NaN,NaN
5,6,Steering,Steering Damper,7412,RANCHO,RS5406,RS5000; Single,"{""0"": ""Category: Steering Damper""}",,"{""automatic"": ""Add to Cart to See Price""}",...,12.6,L1,"Loop Bushing Mount - 5/8"" Bushing ID X 1.5/16""...",Yes,NaN,NaN,NaN,NaN,NaN,NaN
6,7,Steering,Steering Damper,7412,RANCHO,RS5407,RS5000; Single,"{""0"": ""Category: Steering Damper""}",,"{""automatic"": ""Add to Cart to See Price""}",...,7.56,L1,"Loop Bushing Mount - 5/8"" Bushing ID X 1.5/16""...",Yes,NaN,NaN,NaN,NaN,NaN,NaN
7,8,Steering,Steering Damper,7412,RANCHO,RS5409,RS5000,"{""0"": ""Category: Steering Damper""}",,"{""automatic"": ""Add to Cart to See Price""}",...,7.93,LS5,Loop Bushing and Sleeve Mount - 10MM Sleeve ID...,Yes,NaN,NaN,NaN,NaN,NaN,NaN
8,9,Steering,Steering Damper,7412,RANCHO,RS5410,RS5000,"{""0"": ""Category: Steering Damper""}",,"{""automatic"": ""Add to Cart to See Price""}",...,9.92,L1,"Loop Bushing Mount - 5/8"" Bushing ID X 1.5/16""...",Yes,NaN,NaN,NaN,NaN,NaN,NaN
9,10,Steering,Steering Damper,7412,RANCHO,RS5411,RS5000; Single; Bolts To O.E. Mounts,"{""0"": ""Category: Steering Damper""}",,"{""automatic"": ""Add to Cart to See Price""}",...,8.11,L1,"Loop Bushing Mount - 5/8"" Bushing ID X 1.5/16""...",Yes,NaN,NaN,NaN,NaN,NaN,NaN


# 2. Dorman

## 2.1. Vehicle

In [ ]:
import pandas as pd

input_ = pd.read_excel('./crawler.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

# = = = = = = = = = = = = = = =

input_2 = input_.drop_duplicates(['Part Number'],
                                 keep='first',
                                 ignore_index=True)
input_2 = input_2.sort_values(by=['No'],
                              ascending=[True],
                              ignore_index=True)

# = = = = = = = = = = = = = = =

from tqdm import tqdm

for i in tqdm(range(len(input_2)), desc='Progress', ncols=77):
    df_temp = input_[input_['Part Number'] == input_2.loc[i, 'Part Number']].reset_index(drop=True)

    # = = = = = = = = = = = = = = =

    dict_vehicle = {}
    for j in range(len(df_temp)):
        make, model, engine, year = df_temp.loc[j, 'Make'], df_temp.loc[j, 'Model'], df_temp.loc[j, 'Configuration'], int(df_temp.loc[j, 'Year'])

        if make not in dict_vehicle:
            dict_vehicle[make] = {model: {engine: [year]}}
        elif model not in dict_vehicle[make]:
            dict_vehicle[make][model] = {engine: [year]}
        elif engine not in dict_vehicle[make][model]:
            dict_vehicle[make][model][engine] = [year]
        elif year not in dict_vehicle[make][model][engine]:
            dict_vehicle[make][model][engine].append(year)

    # = = = = = = = = = = = = = = =

    list_vehicle = []
    for make in dict_vehicle:
        for model in dict_vehicle[make]:
            list_year = []
            for engine in dict_vehicle[make][model]:
                list_year = list(set(list_year + dict_vehicle[make][model][engine]))
            year_begin = min(list_year)
            year_end = max(list_year)
            list_vehicle.append(f'{make} {model} {year_begin}' if year_begin == year_end else f'{make} {model} {year_begin}-{year_end}')

    input_2.loc[i, 'Vehicle W/o Engine'] = '\n'.join(sorted(list_vehicle))

    # = = = = = = = = = = = = = = =

    list_vehicle = []
    for make in dict_vehicle:
        for model in dict_vehicle[make]:
            for engine in dict_vehicle[make][model]:
                year_begin = min(dict_vehicle[make][model][engine])
                year_end = max(dict_vehicle[make][model][engine])
                list_vehicle.append(f'{make} {model} {year_begin} {engine}'.strip() if year_begin == year_end else f'{make} {model} {year_begin}-{year_end} {engine}'.strip())

    input_2.loc[i, 'Vehicle W/ Engine'] = '\n'.join(sorted(list_vehicle))

# = = = = = = = = = = = = = = =

input_2[['No',
         'Part Number',
         'Vehicle W/o Engine',
         'Vehicle W/ Engine']].to_excel('./vehicle.xlsx', index=False)

input_2

# 3. eBay

## 3.1. Vehicle

### 3.1.1. US、UK、AU、CA

In [15]:
import pandas as pd

input_ = pd.read_excel('./crawler-vehicle.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

# = = = = = = = = = = = = = = =

input_2 = input_.drop_duplicates(['Item Number'],
                                 keep='first',
                                 ignore_index=True)
input_2 = input_2.sort_values(by=['No'],
                              ascending=[True],
                              ignore_index=True)

# = = = = = = = = = = = = = = =

from tqdm import tqdm

for i in tqdm(range(len(input_2)), desc='Progress', ncols=77):
    df_temp = input_[input_['Item Number'] == input_2.loc[i, 'Item Number']].reset_index(drop=True)

    # = = = = = = = = = = = = = = =

    dict_vehicle = {}
    for j in range(len(df_temp)):
        make, model, engine, year = df_temp.loc[j, 'Make'], df_temp.loc[j, 'Model'], df_temp.loc[j, 'Engine'], int(df_temp.loc[j, 'Year'])

        if make not in dict_vehicle:
            dict_vehicle[make] = {model: {engine: [year]}}
        elif model not in dict_vehicle[make]:
            dict_vehicle[make][model] = {engine: [year]}
        elif engine not in dict_vehicle[make][model]:
            dict_vehicle[make][model][engine] = [year]
        elif year not in dict_vehicle[make][model][engine]:
            dict_vehicle[make][model][engine].append(year)

    # = = = = = = = = = = = = = = =

    list_vehicle = []
    for make in dict_vehicle:
        for model in dict_vehicle[make]:
            list_year = []
            for engine in dict_vehicle[make][model]:
                list_year = list(set(list_year + dict_vehicle[make][model][engine]))
            year_begin = min(list_year)
            year_end = max(list_year)
            list_vehicle.append(f'{make} {model} {year_begin}' if year_begin == year_end else f'{make} {model} {year_begin}-{year_end}')

    input_2.loc[i, 'Vehicle W/o Engine'] = '\n'.join(sorted(list_vehicle))

    # = = = = = = = = = = = = = = =

    list_vehicle = []
    for make in dict_vehicle:
        for model in dict_vehicle[make]:
            for engine in dict_vehicle[make][model]:
                year_begin = min(dict_vehicle[make][model][engine])
                year_end = max(dict_vehicle[make][model][engine])
                list_vehicle.append(f'{make} {model} {year_begin} {engine}'.strip() if year_begin == year_end else f'{make} {model} {year_begin}-{year_end} {engine}'.strip())

    input_2.loc[i, 'Vehicle W/ Engine'] = '\n'.join(sorted(list_vehicle))

# = = = = = = = = = = = = = = =

input_2[['No',
         'Item Number',
         'Vehicle W/o Engine',
         'Vehicle W/ Engine']].to_excel('./vehicle.xlsx', index=False)

input_2

Progress: 100%|██████████████████████████████| 80/80 [00:01<00:00, 46.69it/s]


,No,Item Number,Json_Compatibility,Page,Row,Year,Make,Model,Trim,Engine,Notes,Vehicle W/o Engine,Vehicle W/ Engine
0,1,144647596897,"{""api"": ""https://www.ebay.com/g/api/finders"", ...",1,1,2006,Jeep,Wrangler,65th Anniversary Edition Sport Utility 2-Door,4.0L 242Cu. In. l6 GAS OHV Naturally Aspirated,See Tech Notes Below,Jeep Cherokee 1984-2001\nJeep Comanche 1986-19...,Jeep Cherokee 1984-1986 2.8L 173Cu. In. V6 GAS...
1,2,143369543065,"{""api"": ""https://www.ebay.com/g/api/finders"", ...",1,1,2010,Chevrolet,Silverado 2500 HD,LT Crew Cab Pickup 4-Door,6.6L 6599CC 403Cu. In. V8 DIESEL OHV Turbocharged,See Tech Notes Below,Chevrolet Silverado 2500 HD 2001-2010\nChevrol...,Chevrolet Silverado 2500 HD 2001-2006 8.1L 496...
2,3,381979036590,"{""api"": ""https://www.ebay.com/g/api/finders"", ...",1,1,2005,Ford,Excursion,Eddie Bauer Sport Utility 4-Door,5.4L 330Cu. In. V8 GAS SOHC Naturally Aspirated,See Tech Notes Below,Ford Excursion 2000-2005\nFord F-250 Super Dut...,Ford Excursion 2000-2003 7.3L 445Cu. In. V8 DI...
3,4,363910183964,"{""api"": ""https://www.ebay.com/g/api/finders"", ...",1,1,2012,Ram,2500,Hemi Sport Standard Cab Pickup 2-Door,5.7L 345Cu. In. V8 GAS OHV Naturally Aspirated,See Tech Notes Below,Dodge Ram 1500 1994-2001\nDodge Ram 2500 1994-...,Dodge Ram 1500 1994-2001 3.9L 3906CC 239Cu. In...
4,5,143362929011,"{""api"": ""https://www.ebay.com/g/api/finders"", ...",1,1,2012,Ram,2500,Hemi Sport Standard Cab Pickup 2-Door,5.7L 345Cu. In. V8 GAS OHV Naturally Aspirated,See Tech Notes Below,Dodge Ram 1500 1994-2001\nDodge Ram 2500 1994-...,Dodge Ram 1500 1994-2001 3.9L 3906CC 239Cu. In...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,77,386731810217,"{""api"": ""https://www.ebay.com/g/api/finders"", ...",1,1,2000,Chevrolet,C2500,Base Crew Cab Pickup 4-Door,5.7L 350Cu. In. V8 CNG OHV Naturally Aspirated,See Tech Notes Below,Chevrolet C2500 1988-2000\nChevrolet K2500 198...,Chevrolet C2500 1988-1993 6.2L 379Cu. In. V8 D...
76,78,364237130224,"{""api"": ""https://www.ebay.com/g/api/finders"", ...",1,1,2018,Jeep,Wrangler,Rubicon Sport Utility 2-Door,2.0L 1997CC 122Cu. In. l4 MILD HYBRID EV-GAS (...,See Tech Notes Below,Jeep Wrangler 2007-2018,Jeep Wrangler 2007-2011 3.8L 3778CC 231Cu. In....
77,79,145765404617,"{""api"": ""https://www.ebay.com/g/api/finders"", ...",1,1,1998,Ford,F-250,Base Extended Cab Pickup 3-Door,4.6L 281Cu. In. V8 GAS SOHC Naturally Aspirated,See Tech Notes Below,Ford Bronco 1980-1996\nFord F-150 1980-1996\nF...,Ford Bronco 1980-1992 4.9L 4917CC 300Cu. In. l...
78,80,364833212073,"{""api"": ""https://www.ebay.com/g/api/finders"", ...",1,1,1991,Suzuki,Samurai,JA Sport Utility 2-Door,1.3L 1298CC 79Cu. In. l4 GAS SOHC Naturally As...,See Tech Notes Below,Dodge Raider 1987-1989\nJeep CJ5 1965-1983\nJe...,Dodge Raider 1987-1989 2.6L 2555CC 156Cu. In. ...


### 3.1.2. DE

In [ ]:
import pandas as pd

input_ = pd.read_excel('./crawler-vehicle.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

# = = = = = = = = = = = = = = =

input_2 = input_.drop_duplicates(['Item Number'],
                                 keep='first',
                                 ignore_index=True)
input_2 = input_2.sort_values(by=['No'],
                              ascending=[True],
                              ignore_index=True)

# = = = = = = = = = = = = = = =

from tqdm import tqdm

for i in tqdm(range(len(input_2)), desc='Progress', ncols=77):
    df_temp = input_[input_['Item Number'] == input_2.loc[i, 'Item Number']].reset_index(drop=True)

    # = = = = = = = = = = = = = = =

    dict_vehicle = {}
    for j in range(len(df_temp)):
        make, model, engine, year, year_2 = df_temp.loc[j, 'Marke'], df_temp.loc[j, 'Modell'], df_temp.loc[j, 'Motor'], int(df_temp.loc[j, 'Baujahr'].split('-')[0].split('/')[0]), int(df_temp.loc[j, 'Baujahr'].split('-')[1].split('/')[0])

        if make not in dict_vehicle:
            dict_vehicle[make] = {model: {engine: [year] if year == year_2 else [year, year_2]}}
        elif model not in dict_vehicle[make]:
            dict_vehicle[make][model] = {engine: [year] if year == year_2 else [year, year_2]}
        elif engine not in dict_vehicle[make][model]:
            dict_vehicle[make][model][engine] = [year] if year == year_2 else [year, year_2]
        else:
            if year not in dict_vehicle[make][model][engine]:
                dict_vehicle[make][model][engine].append(year)
            if year_2 not in dict_vehicle[make][model][engine]:
                dict_vehicle[make][model][engine].append(year_2)

    # = = = = = = = = = = = = = = =

    list_vehicle = []
    for make in dict_vehicle:
        for model in dict_vehicle[make]:
            list_year = []
            for engine in dict_vehicle[make][model]:
                list_year = list(set(list_year + dict_vehicle[make][model][engine]))
            year_begin = min(list_year)
            year_end = max(list_year)
            list_vehicle.append(f'{make} {model} {year_begin}' if year_begin == year_end else f'{make} {model} {year_begin}-{year_end}')

    input_2.loc[i, 'Vehicle W/o Engine'] = '\n'.join(sorted(list_vehicle))

    # = = = = = = = = = = = = = = =

    list_vehicle = []
    for make in dict_vehicle:
        for model in dict_vehicle[make]:
            for engine in dict_vehicle[make][model]:
                year_begin = min(dict_vehicle[make][model][engine])
                year_end = max(dict_vehicle[make][model][engine])
                list_vehicle.append(f'{make} {model} {year_begin} {engine}'.strip() if year_begin == year_end else f'{make} {model} {year_begin}-{year_end} {engine}'.strip())

    input_2.loc[i, 'Vehicle W/ Engine'] = '\n'.join(sorted(list_vehicle))

# = = = = = = = = = = = = = = =

input_2[['No',
         'Item Number',
         'Vehicle W/o Engine',
         'Vehicle W/ Engine']].to_excel('./vehicle.xlsx', index=False)

input_2

## 3.2 Json

In [14]:
import pandas as pd

input_ = pd.read_excel('./crawler.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import json

for i in tqdm(range(len(input_)), desc='Progress', ncols=77):
    dict_ = json.loads(input_.loc[i, 'Json_Specific'])
    for value in dict_.values():
        input_.loc[i, list(value.keys())[0]] = list(value.values())[0]

input_.to_excel('./crawler.xlsx', index=False)

input_

Progress: 100%|█████████████████████████████| 81/81 [00:00<00:00, 991.53it/s]


,No,Item Number,Title,Subtitle,Price,Sold,Available,Picture,Url,Description Url,...,Drilling,Gas Charged Shock,UPC,Type,Country/Region of Manufacture,Color,Material,Mounting Style,Fitment Type,Warranty
0,1,144647596897,Rough Country OE Replacement Steering Stabiliz...,**Official Rough Country eBay Store**,US $25.95,565 sold,More than 10 available,,https://www.ebay.com/itm/144647596897?_ul=US&_...,https://vi.vipr.ebaydesc.com/itmdesc/144647596...,...,No,Yes,840269912993,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,143369543065,Rough Country N3 Steering Stabilizer for Jeep ...,**Official Rough Country eBay Store**,US $34.95,"2,870 sold",More than 10 available,,https://www.ebay.com/itm/143369543065?_ul=US&_...,https://vi.vipr.ebaydesc.com/itmdesc/143369543...,...,No,Yes,843030147044,Steering Stabilizer,NaN,NaN,NaN,NaN,NaN,NaN
2,3,381979036590,Rough Country N3 Steering Stabilizer for 1999-...,**Official Rough Country eBay Store**,US $44.95,666 sold,More than 10 available,,https://www.ebay.com/itm/381979036590?_ul=US&_...,https://vi.vipr.ebaydesc.com/itmdesc/381979036...,...,No,Yes,843030146733,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,363910183964,Rough Country OE Replacement Steering Stabiliz...,**Official Rough Country eBay Store**,US $25.95,351 sold,More than 10 available,,https://www.ebay.com/itm/363910183964?_ul=US&_...,https://vi.vipr.ebaydesc.com/itmdesc/363910183...,...,No,Yes,840269912986,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,143362929011,Rough Country N3 Steering Stabilizer for 2010-...,**Official Rough Country eBay Store**,US $44.95,616 sold,More than 10 available,,https://www.ebay.com/itm/143362929011?_ul=US&_...,https://vi.vipr.ebaydesc.com/itmdesc/143362929...,...,No,Yes,843030146764,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,77,386731810217,Rough Country V2 Steering Stabilizer for 88-00...,**Official Rough Country eBay Store**,US $114.95,1 sold,6 available,,https://www.ebay.com/itm/386731810217?_ul=US&_...,https://vi.vipr.ebaydesc.com/itmdesc/386731810...,...,No,Yes,840269903601,NaN,NaN,NaN,NaN,NaN,NaN,NaN
77,78,364237130224,Rough Country V2 Steering Stabilizer for 2007-...,**Official Rough Country eBay Store**,US $144.95,2 sold,More than 10 available,,https://www.ebay.com/itm/364237130224?_ul=US&_...,https://vi.vipr.ebaydesc.com/itmdesc/364237130...,...,No,Yes,843030109387,NaN,United States,NaN,NaN,NaN,NaN,NaN
78,79,145765404617,Rough Country V2 Steering Stabilizer for 1980-...,**Official Rough Country eBay Store**,US $114.95,,More than 10 available,,https://www.ebay.com/itm/145765404617?_ul=US&_...,https://vi.vipr.ebaydesc.com/itmdesc/145765404...,...,No,Yes,840269903519,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79,80,364833212073,Rough Country V2 Steering Stabilizer for 1976-...,**Official Rough Country eBay Store**,US $114.95,,More than 10 available,,https://www.ebay.com/itm/364833212073?_ul=US&_...,https://vi.vipr.ebaydesc.com/itmdesc/364833212...,...,No,Yes,840269903533,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 4. Cardone

## 4.1. Vehicle

In [ ]:
import pandas as pd

input_ = pd.read_excel('./crawler.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

# = = = = = = = = = = = = = = =

input_2 = input_.drop_duplicates(['Part Number'],
                                 keep='first',
                                 ignore_index=True)
input_2 = input_2.sort_values(by=['No'],
                              ascending=[True],
                              ignore_index=True)

# = = = = = = = = = = = = = = =

from tqdm import tqdm

for i in tqdm(range(len(input_2)), desc='Progress', ncols=77):
    df_temp = input_[input_['Part Number'] == input_2.loc[i, 'Part Number']].reset_index(drop=True)

    # = = = = = = = = = = = = = = =

    dict_vehicle = {}
    for j in range(len(df_temp)):
        make, model, engine, year = df_temp.loc[j, 'Make'], df_temp.loc[j, 'Model'], ' '.join([df_temp.loc[j, 'Cylinder'], df_temp.loc[j, 'Displacement'], df_temp.loc[j, 'Liter']]), int(df_temp.loc[j, 'Year'])

        if make not in dict_vehicle:
            dict_vehicle[make] = {model: {engine: [year]}}
        elif model not in dict_vehicle[make]:
            dict_vehicle[make][model] = {engine: [year]}
        elif engine not in dict_vehicle[make][model]:
            dict_vehicle[make][model][engine] = [year]
        elif year not in dict_vehicle[make][model][engine]:
            dict_vehicle[make][model][engine].append(year)

    # = = = = = = = = = = = = = = =

    list_vehicle = []
    for make in dict_vehicle:
        for model in dict_vehicle[make]:
            list_year = []
            for engine in dict_vehicle[make][model]:
                list_year = list(set(list_year + dict_vehicle[make][model][engine]))
            year_begin = min(list_year)
            year_end = max(list_year)
            list_vehicle.append(f'{make} {model} {year_begin}' if year_begin == year_end else f'{make} {model} {year_begin}-{year_end}')

    input_2.loc[i, 'Vehicle W/o Engine'] = '\n'.join(sorted(list_vehicle))

    # = = = = = = = = = = = = = = =

    list_vehicle = []
    for make in dict_vehicle:
        for model in dict_vehicle[make]:
            for engine in dict_vehicle[make][model]:
                year_begin = min(dict_vehicle[make][model][engine])
                year_end = max(dict_vehicle[make][model][engine])
                list_vehicle.append(f'{make} {model} {year_begin} {engine}'.strip() if year_begin == year_end else f'{make} {model} {year_begin}-{year_end} {engine}'.strip())

    input_2.loc[i, 'Vehicle W/ Engine'] = '\n'.join(sorted(list_vehicle))

# = = = = = = = = = = = = = = =

input_2[['No',
         'Part Number',
         'Vehicle W/o Engine',
         'Vehicle W/ Engine']].to_excel('./vehicle.xlsx', index=False)

input_2

## 4.2. Json

In [ ]:
# input_ = pd.read_excel('crawler.xlsx',
#                        header=0,
#                        dtype=str).fillna('')

# input_

# import json

# for i in tqdm(range(len(input_)), desc='Progress', ncols=77):
#     dict_ = json.loads(input_.loc[i, 'Json_Info'])
#     for value in dict_.values():
#         input_.loc[i, list(value.keys())[0]] = list(value.values())[0]

# input_.to_excel('./json.xlsx', index=False)

# input_

# 7. SpectraPremium

## 7.1. Vehicle

In [ ]:
import pandas as pd

input_ = pd.read_excel('./crawler.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

# = = = = = = = = = = = = = = =

input_2 = input_.drop_duplicates(['Part Number'],
                                 keep='first',
                                 ignore_index=True)
input_2 = input_2.sort_values(by=['No'],
                              ascending=[True],
                              ignore_index=True)

# = = = = = = = = = = = = = = =

from tqdm import tqdm

for i in tqdm(range(len(input_2)), desc='Progress', ncols=77):
    df_temp = input_[input_['Part Number'] == input_2.loc[i, 'Part Number']].reset_index(drop=True)

    # = = = = = = = = = = = = = = =

    dict_vehicle = {}
    for j in range(len(df_temp)):
        make_model_engine, year = df_temp.loc[j, 'Make Model Engine'], int(df_temp.loc[j, 'Year'])

        if make_model_engine not in dict_vehicle:
            dict_vehicle[make_model_engine] = [year]
        elif year not in dict_vehicle[make_model_engine]:
            dict_vehicle[make_model_engine].append(year)

    # = = = = = = = = = = = = = = =

    list_vehicle = []
    for make_model_engine in dict_vehicle:
        year_begin = min(dict_vehicle[make_model_engine])
        year_end = max(dict_vehicle[make_model_engine])
        list_vehicle.append(f'{make_model_engine}'.strip() if year_begin == year_end else f'{make_model_engine} {year_begin}-{year_end}'.strip())

    input_2.loc[i, 'Vehicle'] = '\n'.join(sorted(list_vehicle))

# = = = = = = = = = = = = = = =

input_2[['No',
         'Part Number',
         'Vehicle']].to_excel('./vehicle.xlsx', index=False)

input_2

# 30.TecDoc

## 30.1. Vehicle

In [ ]:
import pandas as pd

input_ = pd.read_excel('./crawler.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

# = = = = = = = = = = = = = = =

input_2 = input_.drop_duplicates(['Part Number'],
                                 keep='first',
                                 ignore_index=True)
input_2 = input_2.sort_values(by=['No'],
                              ascending=[True],
                              ignore_index=True)

# = = = = = = = = = = = = = = =

from tqdm import tqdm
from datetime import datetime

for i in tqdm(range(len(input_2)), desc='Progress', ncols=77):
    df_temp = input_[input_['Part Number'] == input_2.loc[i, 'Part Number']].reset_index(drop=True)

    # = = = = = = = = = = = = = = =

    dict_vehicle = {}
    for j in range(len(df_temp)):
        make, model, engine, year, year_2 = df_temp.loc[j, 'manuDesc'], df_temp.loc[j, 'modelDesc'], df_temp.loc[j, 'carDesc'], int(str(df_temp.loc[j, 'yearOfConstructionFrom'][:4])), int(datetime.now().strftime('%Y') if df_temp.loc[j, 'yearOfConstructionTo'] == '' else str(df_temp.loc[j, 'yearOfConstructionTo'][:4]))

        if make not in dict_vehicle:
            dict_vehicle[make] = {model: {engine: [year] if year == year_2 else [year, year_2]}}
        elif model not in dict_vehicle[make]:
            dict_vehicle[make][model] = {engine: [year] if year == year_2 else [year, year_2]}
        elif engine not in dict_vehicle[make][model]:
            dict_vehicle[make][model][engine] = [year] if year == year_2 else [year, year_2]
        else:
            if year not in dict_vehicle[make][model][engine]:
                dict_vehicle[make][model][engine].append(year)
            if year_2 not in dict_vehicle[make][model][engine]:
                dict_vehicle[make][model][engine].append(year_2)

    # = = = = = = = = = = = = = = =

    list_vehicle = []
    for make in dict_vehicle:
        for model in dict_vehicle[make]:
            list_year = []
            for engine in dict_vehicle[make][model]:
                list_year = list(set(list_year + dict_vehicle[make][model][engine]))
            year_begin = min(list_year)
            year_end = max(list_year)
            list_vehicle.append(f'{make} {model} {year_begin}' if year_begin == year_end else f'{make} {model} {year_begin}-{year_end}')

    input_2.loc[i, 'Vehicle W/o Engine'] = '\n'.join(sorted(list_vehicle))

    # = = = = = = = = = = = = = = =

    list_vehicle = []
    for make in dict_vehicle:
        for model in dict_vehicle[make]:
            for engine in dict_vehicle[make][model]:
                year_begin = min(dict_vehicle[make][model][engine])
                year_end = max(dict_vehicle[make][model][engine])
                list_vehicle.append(f'{make} {model} {year_begin} {engine}'.strip() if year_begin == year_end else f'{make} {model} {year_begin}-{year_end} {engine}'.strip())

    input_2.loc[i, 'Vehicle W/ Engine'] = '\n'.join(sorted(list_vehicle))

# = = = = = = = = = = = = = = =

input_2[['No',
         'Part Number',
         'Vehicle W/o Engine',
         'Vehicle W/ Engine']].to_excel('./vehicle.xlsx', index=False)

input_2

# 10. Amazon

In [ ]:
import pandas as pd

input_ = pd.read_excel('./crawler.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_

In [ ]:
import json

for i in range(len(input_)):
    dict_ = json.loads(input_.loc[i, 'Json_Overview'])
    for value in dict_.values():
        input_.loc[i, list(value.keys())[0]] = list(value.values())[0]

    dict_ = json.loads(input_.loc[i, 'Json_Detail 1'])
    for value in dict_.values():
        input_.loc[i, list(value.keys())[0]] = list(value.values())[0]

    dict_ = json.loads(input_.loc[i, 'Json_Detail 2'])
    for value in dict_.values():
        input_.loc[i, list(value.keys())[0]] = list(value.values())[0]

    dict_ = json.loads(input_.loc[i, 'Json_Information 1'])
    for value in dict_.values():
        if list(value.keys())[0] == 'Customer Reviews':
            input_.loc[i, f'{list(value.keys())[0]} Star'] = list(value.values())[0]['0']
            input_.loc[i, f'{list(value.keys())[0]} Rating'] = list(value.values())[0]['1']
        elif list(value.keys())[0] == 'Best Sellers Rank':
            list_rank = []
            for value_2 in list(value.values())[0].values():
                list_rank.append(f'{list(value_2.keys())[0]} -> {list(value_2.values())[0]}')
            input_.loc[i, 'Best Sellers Rank'] = '\n'.join(list_rank)
        else:
            input_.loc[i, list(value.keys())[0]] = list(value.values())[0]

    dict_ = json.loads(input_.loc[i, 'Json_Information 2'])
    for value in dict_.values():
        if list(value.keys())[0] == 'Customer Reviews':
            input_.loc[i, f'{list(value.keys())[0]} Star'] = list(value.values())[0]['0']
            input_.loc[i, f'{list(value.keys())[0]} Rating'] = list(value.values())[0]['1']
        elif list(value.keys())[0] == 'Best Sellers Rank':
            list_rank = []
            for value_2 in list(value.values())[0].values():
                list_rank.append(f'{list(value_2.keys())[0]} -> {list(value_2.values())[0]}')
            input_.loc[i, 'Best Sellers Rank'] = '\n'.join(list_rank)
        else:
            input_.loc[i, list(value.keys())[0]] = list(value.values())[0]

input_.to_excel('./fuck.xlsx', index=False)

input_